### Trying to solve Task 4 now!

In [1]:
random_state = 42
EPOCHS = 200
BATCH_SIZE = 128

In [2]:
!pip install sklearn numpy matplotlib psutil pandas

     |████████████████████████████████| 291 kB 11.2 MB/s eta 0:00:01
     |████████████████████████████████| 9.5 MB 16.0 MB/s eta 0:00:01
     |████████████████████████████████| 22.2 MB 121.4 MB/s eta 0:00:01
     |████████████████████████████████| 510 kB 112.8 MB/s eta 0:00:01
     |████████████████████████████████| 25.9 MB 107.2 MB/s eta 0:00:01
     |████████████████████████████████| 303 kB 121.8 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=25ac3524309e5f6599e8b8cd0f513cdbd898f295ade4a54a195a806ac9a2555b
  Stored in directory: /root/.cache/pip/wheels/23/9d/42/5ec745cbbb17517000a53cecc49d6a865450d1f5cb16dc8a9c
Successfully built sklearn
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
!pip install -U tensorboard_plugin_profile

     |████████████████████████████████| 1.2 MB 11.1 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
%load_ext tensorboard

In [5]:
%tensorboard --logdir logs/ --bind_all

In [6]:
# run for a remote notebook: docker run -it --gpus all -v /root/iml2021/task4:/tf/task4 -p 8888:8888 tensorflow/tensorflow:latest-gpu-jupyter
# need stuff cd /task4 && python -m pip install sklearn numpy matplotlib psutil pandas && python3 task4.py
from tensorflow.keras.applications import ResNet50
#from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
#from tensorflow.keras.applications.mobilenet import preprocess_input, decode_predictions
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.data import Dataset
from tensorflow import keras
import tensorflow as tf

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os
import itertools
import matplotlib.pyplot as plt
from lru_cache_memory_aware import lru_cache
from datetime import datetime
from packaging import version
import multiprocessing

Following is the example by keras.

In [7]:
# TODO https://keras.io/api/applications/resnet/ use ResNet50v2 and then one last time ResNet152v2
def example():
  model = ResNet50(
      include_top=True,
      weights="imagenet",
      input_tensor=None,
      input_shape=None,
      pooling=None,
      classes=1000,
      classifier_activation="softmax",
  )
  img_path = 'food/00000.jpg'
  img = image.load_img(img_path, target_size=(224, 224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)

  preds = model.predict(x)
  # decode the results into a list of tuples (class, description, probability)
  # (one such list for each sample in the batch)
  print('Predicted:', decode_predictions(preds, top=3)[0])

In [8]:
%%capture
!unzip -o food.zip

In [9]:
trainPD = pd.read_csv("train_triplets.txt", header=None, delimiter=' ')
testPD = pd.read_csv("test_triplets.txt", header=None, delimiter=' ')
# Add the y lable, where 1 means exactly what the exercise defines,
# i.e. 0 is closer in taste to 1 than 2
trainPDreverse = trainPD.copy()
trainPDreverse[1] = trainPD[2].copy()
trainPDreverse[2] = trainPD[1].copy()
trainPD['y'] = pd.DataFrame(np.ones((trainPD.shape[0],1)), columns=["y"])
trainPDreverse['y'] = pd.DataFrame(np.zeros((trainPDreverse.shape[0],1)), columns=["y"])
trainPD = trainPD.append(trainPDreverse).reset_index(drop=True)
# TODO move B to A -> same lable, but a should make for more diverse input with same idea behind it
trainPDreverseF = trainPD.copy()
trainPDreverseF[0] = trainPD[1].copy()
trainPDreverseF[1] = trainPD[0].copy()
trainPD = trainPD.append(trainPDreverseF).reset_index(drop=True)
# Give labels and reshuffle
trainPD.columns = ['A', 'B', 'C', 'y']
trainPD = trainPD.sample(frac=1, random_state=random_state).reset_index(drop=True)
trainPD

,A,B,C,y
0,2762,3127,1659,0.0
1,1531,62,3849,1.0
2,1514,1885,373,1.0
3,2601,4295,845,1.0
4,1306,3411,4212,1.0
...,...,...,...,...
238055,3177,1791,1130,1.0
238056,714,1913,3094,0.0
238057,2801,62,1828,1.0
238058,709,1363,1010,1.0


Prepare the data splits  
TODO: create the generator with turning and then combine the correct 3 always.

In [10]:
# split picture combinations
def createTrainSplit(df):
  global train_split, test_split, val_split
  train_split, test_split = train_test_split(df, test_size=0.1, random_state=random_state)
  #train_split, val_split = train_test_split(train_split, test_size=0.2, random_state=random_state)

createTrainSplit(trainPD)
print(train_split.shape)
train_split.head(2)

(214254, 4)


,A,B,C,y
60764,2640,3419,4488,0.0
103755,4877,851,2256,0.0


Let's do the preprocessing

In [11]:
# TODO maybe think if actually using resnet preprocess_input is ok
trainAug = ImageDataGenerator(
    rotation_range=25,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    preprocessing_function=preprocess_input)

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator(preprocessing_function=preprocess_input)

SHIT we have to write our own generator because of the different input... [example](https://medium.com/analytics-vidhya/write-your-own-custom-data-generator-for-tensorflow-keras-1252b64e41c3)

In [12]:
def multiply_df(amount, df:pd.DataFrame):
  ret = df.copy()
  ret = ret.apply(lambda row: row.loc[row.index.repeat(amount)], axis=0).reset_index(drop=True)
  # now add the transform_n just to see in index afterwards
  ns = np.concatenate(list(itertools.repeat(np.arange(0,amount), df.shape[0])))
  ret["transform_n"] = pd.DataFrame(ns,columns=["transform_n"])
  return ret

multiply_df(2, train_split)

,A,B,C,y,transform_n
0,2640,3419,4488,0.0,0
1,2640,3419,4488,0.0,1
2,4877,851,2256,0.0,0
3,4877,851,2256,0.0,1
4,4068,367,1567,1.0,0
...,...,...,...,...,...
428503,969,720,191,0.0,1
428504,3243,4276,3461,1.0,0
428505,3243,4276,3461,1.0,1
428506,2881,4180,3300,1.0,0


In [13]:
def show_images(images: [np.ndarray]) -> None:
    n: int = len(images)
    f = plt.figure()
    for i in range(n):
        # Debug, plot figure
        f.add_subplot(1, n, i + 1)
        plt.imshow(image.array_to_img(images[i]))

    plt.show(block=True)

In [14]:
def load_image(image_index):
    path = f"food/{str(image_index).zfill(5)}.jpg"
    h, w, = (224, 224)
    img = image.load_img(path, target_size=(h, w))
    img = image.img_to_array(img)
    return img

In [15]:
imageDict = {}
for i in range(0, 10000):
    if i % 1000 == 0:
        print(i)
    imageDict[i] = load_image(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [16]:
try:
    cpus = multiprocessing.cpu_count()
except NotImplementedError:
    cpus = 2   # arbitrary default
pool = multiprocessing.Pool(processes=cpus)

Process ForkPoolWorker-7:
Process ForkPoolWorker-5:
Process ForkPoolWorker-1:
Process ForkPoolWorker-2:
Process ForkPoolWorker-3:
Process ForkPoolWorker-8:
Process ForkPoolWorker-6:
Process ForkPoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/proces

In [17]:
class task4DataGenerator(keras.utils.Sequence):

  def __init__(self, df,
                batch_size=1024,
                input_size=(224, 224, 3),
                augmenter:ImageDataGenerator=None,
                shuffle=False,
                seed=42,
                amount_augmented=1,
                do_multiply=False,
                do_augment=False,
                ret_labels=True
               ):
        
    self.df = multiply_df(amount_augmented, df) if do_multiply else df
    self.batch_size = batch_size
    self.input_size = input_size
    self.shuffle = shuffle
    self.seed = seed
    self.do_augment = do_augment
    self.augmenter:ImageDataGenerator = augmenter
    self.amount_augmented = amount_augmented
    self.ret_labels = ret_labels

    self.n = len(self.df)

  # GB = 1024**3
  #@lru_cache(use_memory_up_to=(leave_free_GB * 1024**3))
  #def __load_image(self, image_index):
  #  path = f"food/{str(image_index).zfill(5)}.jpg"
  #  h, w, chann = self.input_size
  #  img = image.load_img(path, target_size=(h, w))
  #  img = image.img_to_array(img)
  #  return img

  def __get_image_batch(self, img_indices):
    batch = [imageDict[x] for _, x in img_indices.iteritems()]
    batch = np.stack( batch, axis=0 )
    return batch

  def __get_transform(self, img_batch):
    # returns the transformed img_batch if augmenter defined
    # TODO check if the random_transform returns a copy or else 
    # use https://stackoverflow.com/questions/54909357/how-to-get-functools-lru-cache-to-return-new-instances
    # to return copy of load_img
    # TODO self seed probably does not do it here! , seed=self.seed
    #img_batch = [self.augmenter.random_transform(img, seed=self.seed) for img in img_batch], axis=0 )
    if self.do_augment:
    #    img_batch = pool.map(self.augmenter.random_transform, img_batch)
    #    img_batch = np.stack(img_batch, axis=0)
        for i, img in enumerate(img_batch):
            img_batch[i] = self.augmenter.random_transform(img, seed=self.seed)
    img_batch = self.augmenter.standardize(img_batch)
    return img_batch

  def __get_data(self, batch_df):
    
    # batch_df is of form [A, B, C, y, transform_n] x n
    batch_A = self.__get_image_batch(batch_df['A'])
    batch_B = self.__get_image_batch(batch_df['B'])
    batch_C = self.__get_image_batch(batch_df['C'])
    #show_images(batch_A)
    batch_A = self.__get_transform(batch_A)
    #show_images(batch_A)
    batch_B = self.__get_transform(batch_B)
    batch_C = self.__get_transform(batch_C)
    X = [batch_A, batch_B, batch_C]
    y = batch_df['y'].to_numpy() if self.ret_labels else []
    return X, y

  def on_epoch_end(self):
    if self.shuffle:
      self.df = self.df.sample(frac=1, random_state=self.seed).reset_index(drop=True)
  
  def __getitem__(self, index):
    batches = self.df[index * self.batch_size:(index + 1) * self.batch_size]
    X, y = self.__get_data(batches)
    return X, y
  
  def __len__(self):
    return self.n // self.batch_size

testDG = task4DataGenerator(train_split, batch_size=2, augmenter=trainAug)
X, y = testDG[0]

In [18]:
trainGN = task4DataGenerator(train_split, do_augment=True, do_multiply=True, 
                             augmenter=trainAug, batch_size=BATCH_SIZE,
                            shuffle=True)
valGN = task4DataGenerator(test_split, augmenter=valAug, batch_size=BATCH_SIZE)
# let's use the public score as test actually
#testGN = task4DataGenerator(test_split, augmenter=valAug, batch_size=BATCH_SIZE)

Let's try to follow [this](https://www.pyimagesearch.com/2020/04/27/fine-tuning-resnet-with-keras-tensorflow-and-deep-learning/) example for a bit in regards to fine tuning the ResNet network

In [19]:
# TODO https://keras.io/api/applications/resnet/ use ResNet50v2 and then one last time ResNet152v2
def getBaseModelRN50(input_shape=(224, 224, 3)):
  # ImageNet trained ResNet50
  baseModel = ResNet50(
      include_top=False,
      weights="imagenet",
      input_tensor=None,
      input_shape=input_shape,
      pooling=None,
      classifier_activation="softmax",
  )
  for layer in baseModel.layers:
    layer.trainable = False
  return baseModel

In [20]:
def getMobileNet(input_shape=(224, 224, 3)):
  baseModel = MobileNet(
    input_shape=input_shape,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classifier_activation="softmax")
  for layer in baseModel.layers:
    layer.trainable = False
  return baseModel

In [21]:
def getSingleImgModel50(baseModel, inp):
  # The following builds a new head for the baseModel to get the flatten
  headModel = baseModel(inp)
  #headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
  headModel = AveragePooling2D(pool_size=(4, 4))(headModel) # upper or this
  headModel = Flatten(name="flatten")(headModel)
  #headModel = Dense(512, activation="relu")(headModel)
  # place the head model on top of the base model (this will become
  # the model for each picture we use)
  return Model(inputs=inp, outputs=headModel)

In [22]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_model(metrics=METRICS, output_bias=None):
  #baseModel = None
  baseModel = getBaseModelRN50()
  input_A = keras.Input(shape=(224, 224, 3), name='input_A')
  input_B = keras.Input(shape=(224, 224, 3), name='input_B')
  input_C = keras.Input(shape=(224, 224, 3), name='input_C')
  modelA = getSingleImgModel50(baseModel, input_A)
  modelB = getSingleImgModel50(baseModel, input_B)
  modelC = getSingleImgModel50(baseModel, input_C)
  #Fix layer naming as otherwise we have colissions
  for layer in modelA.layers :
    if "input" not in layer._name:
        layer._name = layer.name + str('_A')
  for layer in modelB.layers :
    if "input" not in layer._name:
        layer._name = layer.name + str('_B')
  for layer in modelC.layers :
    if "input" not in layer._name:
        layer._name = layer.name + str('_C')

  # we use 3 times ImageNet to get the features from the 3 images combined
  concat = Concatenate()([modelA.outputs[0], modelB.outputs[0], modelC.outputs[0]])
  headModel = Dropout(0.5)(concat)
  headModel = Dense(512, activation="relu")(headModel)
  headModel = Dropout(0.5)(headModel)
  headModel = Dense(512, activation="relu")(headModel)
  headModel = Dropout(0.5)(headModel)
  #headModel = Dense(512, activation="relu")(headModel)
  #headModel = Dropout(0.5)(headModel)
  #headModel = Dense(128, activation="relu")(headModel)
  #headModel = Dropout(0.25)(headModel)
  headModel = Dense(1, activation='sigmoid')(headModel)
  # try https://github.com/noelcodella/tripletloss-keras-tensorflow/blob/master/tripletloss.py
  model = Model(inputs=[input_A, input_B, input_C], outputs=[headModel])

  # actually compile it now
  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.binary_crossentropy,
      metrics=metrics)

  return model

In [23]:
make_model().summary()

94773248/94765736 [==============================] - 2s 0us/step
Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_A (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_B (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
input_C (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
resnet50_A_B_C (Functional)     (None, 7, 7, 2048)   23587712    input_A[0][0]                    
                           

In [24]:
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [25]:
logs = "logs/" + datetime.now().strftime("%Y%m%d-%H%M%S")

#tboard_callback = keras.callbacks.TensorBoard(log_dir = logs, update_freq=50)
tboard_callback = keras.callbacks.TensorBoard(
    log_dir = logs,
    histogram_freq = 1,
    profile_batch=(10,40))

In [26]:
checkpoint_filepath = 'checkpoint/'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1)

In [27]:
print("Making model")
model = make_model()
model.load_weights(checkpoint_filepath)
print("Starting to fit")
fitted = model.fit(
    trainGN,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping, tboard_callback, model_checkpoint_callback],
    validation_data=valGN,
    use_multiprocessing = True,
    workers=8,
    verbose=1)
print("Done fitting")

Making model
Starting to fit


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 1/200
 256/1673 [===>..........................] - ETA: 21:50 - loss: 0.6643 - tp: 5319.0000 - fp: 2700.0000 - tn: 13491.0000 - fn: 11258.0000 - accuracy: 0.5740 - precision: 0.6633 - recall: 0.3209 - auc: 0.6158 - prc: 0.6353

Process Keras_worker_ForkPoolWorker-12:
Process Keras_worker_ForkPoolWorker-9:
Process Keras_worker_ForkPoolWorker-10:
Traceback (most recent call last):
Traceback (most recent call last):
Process Keras_worker_ForkPoolWorker-13:
Process Keras_worker_ForkPoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/pool.py", line 125, in worker
    put((job, i, result))
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Process Keras_worker_ForkPool

KeyboardInterrupt
  File "<ipython-input-17-7a2e34464e89>", line 38, in __get_image_batch
    batch = np.stack( batch, axis=0 )
  File "/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/affine_transformations.py", line 333, in <listcomp>
    cval=cval) for x_channel in x]
  File "<__array_function__ internals>", line 6, in stack
Process Keras_worker_ForkPoolWorker-11:
  File "/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py", line 528, in affine_transform
    output, order, mode, cval, None, None)
  File "/usr/local/lib/python3.6/dist-packages/numpy/core/shape_base.py", line 434, in stack
    return _nx.concatenate(expanded_arrays, axis=axis, out=out)
KeyboardInterrupt
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
KeyboardInterrupt
  File "<__array_function__ internals>", line 6, in concatenate
KeyboardInterrupt
  File "/usr/lib/python3.6/multiprocessing/process.py", 

KeyboardInterrupt: 

Check that checkpointing acutally works:

In [31]:
model = make_model()
model.evaluate(valGN, verbose=1, workers=6)

185/185 [==============================] - 170s 895ms/step - loss: 0.7830 - tp: 7420.0000 - fp: 4793.0000 - tn: 23329.0000 - fn: 21034.0000 - accuracy: 0.5435 - precision: 0.6075 - recall: 0.2608 - auc: 0.5585 - prc: 0.5838


[0.7830327153205872,
 7420.0,
 4793.0,
 23329.0,
 21034.0,
 0.5434989929199219,
 0.6075493097305298,
 0.260771781206131,
 0.5584668517112732,
 0.5838417410850525]

In [33]:
model.load_weights(checkpoint_filepath)
model.evaluate(valGN, verbose=1, workers=6)

185/185 [==============================] - 164s 884ms/step - loss: 0.6433 - tp: 5723.0000 - fp: 2656.0000 - tn: 9209.0000 - fn: 6092.0000 - accuracy: 0.6306 - precision: 0.6830 - recall: 0.4844 - auc: 0.6829 - prc: 0.6829


[0.643280565738678,
 5723.0,
 2656.0,
 9209.0,
 6092.0,
 0.6305743455886841,
 0.6830170750617981,
 0.4843842685222626,
 0.68292635679245,
 0.6829150319099426]

Now let's actually create the output for the submission

In [41]:
testPD.columns = ['A', 'B', 'C']
submissionGN = task4DataGenerator(testPD, augmenter=valAug, batch_size=8, ret_labels=False)
testPD

,A,B,C
0,9896,9640,9177
1,6592,9283,7104
2,8655,6174,6400
3,9223,8187,8678
4,7317,5392,9470
...,...,...,...
59539,6113,8042,6277
59540,8851,6075,8549
59541,6299,7843,7940
59542,7652,5620,5416


In [29]:
test = make_model()
test.load_weights(checkpoint_filepath)

In [30]:
res = test.predict(submissionGN, verbose=1, use_multiprocessing=True, workers=6)

7443/7443 [==============================] - 417s 55ms/step


In [36]:
res = pd.DataFrame(res)
res.value_counts()

0.498339    18517
0.412985        6
0.275147        5
0.208707        4
0.696023        4
            ...  
0.556275        1
0.556268        1
0.556251        1
0.556246        1
0.097741        1
Length: 40776, dtype: int64

In [40]:
res

,0
0,0.498339
1,0.279280
2,0.498339
3,0.428493
4,0.628882
...,...
59539,0.227854
59540,0.364465
59541,0.498339
59542,0.498339


This looks strongly like my model has no idea what to do with half of the inputs...

In [42]:
print('Change shape and threshold to 0 1 for submission')
seq_predictions=np.transpose(res.to_numpy())[0]  # transformation to get (n,)
print(seq_predictions.shape)  # now the shape is (n,)
# Applying transformation to get binary values predictions with 0.5 as thresold
seq_predictions = list(map(lambda x: 0 if x<0.5 else 1, seq_predictions))
seq_predictions = pd.DataFrame(seq_predictions)
seq_predictions

Change shape and threshold to 0 1 for submission
(59544,)


,0
0,0
1,0
2,0
3,0
4,1
...,...
59539,0
59540,0
59541,0
59542,0


In [43]:
seq_predictions.to_csv("outThreeModel.csv", index=False, header=False)